In [1]:
import cv2 #Used for image manipulation
import matplotlib.pyplot as plt #Used for testing purposes
import numpy as np #Used for sin, cos, pi
import random as r #Used for seeds, random path generation
import networkx as nx #Used for calculating shortest path
from tqdm import tqdm #Used for progress bars
import time as t #Used to get amount of time algorithm has run

In [2]:
def scale_for_maze(img_name, requested_size):
    #Step 1 - Get a grayscale representation of the input image
    img = cv2.imread("input/" + img_name)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #Step 2 - Shrink the image to something more managable
    max_dist = max(len(gray_img[0]), len(gray_img))
    if requested_size <= max_dist:
        scale = max_dist//requested_size
        new_size = (len(gray_img[0])//scale, len(gray_img)//scale)
    else:
        new_size = gray_img.shape
        
    #Step 3 - Scale the images
    scaled_img = cv2.resize(img, new_size)
    scaled_gray_img = cv2.resize(gray_img, new_size)
    
    return scaled_img, scaled_gray_img

In [3]:
def get_new_path_nodes(in_node, mx_size, graph_info_ar):
    mxx, mxy = mx_size
    inx, iny = in_node
    out_nodes = []
    if inx > 0:
        out_nodes.append((inx-1, iny))
    if iny > 0:
        out_nodes.append((inx, iny-1))
    if inx < mxx - 1:
        out_nodes.append((inx+1, iny))
    if iny < mxy - 1:
        out_nodes.append((inx, iny+1))
    
    valid_nodes = []
    for x, y in out_nodes:
        #If this node hasn't been excluded or visited
        if graph_info_ar[x][y] == 0:
            valid_nodes.append((x, y))
    return valid_nodes

In [4]:
def generate_path(scaled_gray_img, exclude_threshold, noise=0, kernel_size=(3, 3),
                  stack_size_thres=-1, selections=100, seed="Maze Generator", print_path_prog=True):
    r.seed(seed)
    scaled_blur_img = cv2.GaussianBlur(scaled_gray_img, kernel_size, cv2.BORDER_DEFAULT)
    
    size_x, size_y = [len(scaled_blur_img), len(scaled_blur_img[0])]
    nodes_x, nodes_y = [size_x//2 - 1, size_y//2 - 1]
    
    #Get locations of nodes and edges to exclude from the maze
    graph_info_ar = np.zeros((nodes_x, nodes_y))
    num_excluded = 0
    for x in range(nodes_x):
        for y in range(nodes_y):
            if scaled_blur_img[x*2+1][y*2+1] < exclude_threshold:
                graph_info_ar[x][y] = -1 #If a node is excluded, set it's value to -1
                num_excluded += 1
    
    #Start at a valid position and get that position's weight
    x, y = (-1, -1)
    attempts = 0
    while (x, y) == (-1, -1) or graph_info_ar[x][y] == -1:
        if attempts < 100:
            x, y = (r.randint(0, nodes_x-1), 0) if r.randint(0, 1) == 0 else (0, r.randint(0, nodes_y-1))
        else:
            print("Unable to place map entrance on edge of map. Placing in a random location instead.")
            x, y = (r.randint(0, nodes_x-1), r.randint(0, nodes_y-1))
        attempts += 1
            
    stack = [(x, y)]
    weights = [scaled_blur_img[x][y] + 1]
    
    #Start with the first node and it's weight
    visited = [(stack[0][0], stack[0][1])]    
    
    #Start with an empty graph
    maze_graph = nx.Graph()
    
    #Generate the maze
    max_possible_nodes = nodes_x*nodes_y - num_excluded + 1
    
    #Choose parameters for increasing runspeed when the stack is very large
    selections = min(selections, stack_size_thres//4)
    
    while stack:
        #Choose which node to pop
        to_pop = r.choices(range(len(stack)), weights=weights, k=1) 
        
        #If we're ignoring certain weights to increase runspeed
        if stack_size_thres < len(stack) and stack_size_thres > 0:
            #Get several nodes
            to_pop_list = r.sample(range(to_pop[0] - stack_size_thres//2 + 1, to_pop[0]), k=selections)
        else:
            #Otherwise, only work with the first node
            to_pop_list = to_pop
        
        #For every node we're expanding out
        for next_pop in to_pop_list:
            #Remove the current node and weight
            curr_node = stack.pop(next_pop)
            weights.pop(next_pop)
            #For every new connection between two nodes
            for out_x, out_y in get_new_path_nodes(curr_node, (nodes_x, nodes_y), graph_info_ar):
                maze_graph.add_edge(curr_node, (out_x, out_y))    #Create a new edge between 2 nodes
                stack.append((out_x, out_y))                      #Add the new node to the stack
                weights.append(scaled_blur_img[out_x][out_y] + 1) #Get and add the weight of the node
                visited.append((out_x, out_y))                    #Set the new node as visited (list)
                graph_info_ar[out_x][out_y] = 1                   #Set the new node as visited (2D array)
                if len(visited) % 1000 == 0 and print_path_prog:
                    print("Visited", len(visited), "nodes of ", max_possible_nodes, "possibly valid nodes.")
    print(len(visited), "of", max_possible_nodes, "nodes were reachable by the algorithm.")
        
    #Generate noise
    if 0 < noise <= 1:
        for x, y in tqdm(visited, desc="Generating Noise..."):
            #If we're not in the last row or column
            if x + 1 < nodes_x and y + 1 < nodes_y:
                #If we've visited the other node and pass a random check
                if graph_info_ar[x+1][y] == 1 and noise > r.random():
                    maze_graph.add_edge((x, y), (x+1, y))
                if graph_info_ar[x][y+1] == 1 and noise > r.random():
                    maze_graph.add_edge((x, y), (x, y+1))
                
    #Generate the solution path from the first node to the last node
    soltn_nodes = nx.shortest_path(maze_graph, source=visited[0], target=visited[-1])
    
    edges = maze_graph.edges(data=False)
    return visited, edges, soltn_nodes

In [5]:
def get_maze_images(scaled_img, nodes, edges, soltn_nodes):
    size_x, size_y = [len(scaled_img), len(scaled_img[0])]
    nodes_x, nodes_y = [size_x//2 - 1, size_y//2 - 1]
    
    maze_mask = np.zeros((size_x, size_y),dtype=np.uint8)
    #Get starting and ending position
    start, end = [soltn_nodes[0], soltn_nodes[-1]]
    
    #Display all nodes in the maze
    for x, y in tqdm(nodes, desc="Writing Nodes..."):
        maze_mask[x*2+1][y*2+1] = 255
    
    #Display all edges in the maze
    for in_node, out_node in tqdm(edges,desc="Writing edges..."):
        mid_x, mid_y = [(in_node[0] + out_node[0]), (in_node[1] + out_node[1])]
        maze_mask[mid_x+1][mid_y+1] = 255

    #Convert to color image
    maze_plain = cv2.cvtColor(maze_mask, cv2.COLOR_GRAY2BGR)
    mm_b, mm_g, mm_r = cv2.split(maze_plain)        

    ms_b, ms_g, ms_r = cv2.split(maze_plain)
    
    #Display solution path including both nodes and edges
    print("Finding Best Solution...")
    for i in range(len(soltn_nodes) - 1):
        #Get first and second node
        x1, y1 = (soltn_nodes[i][0]*2 + 1, soltn_nodes[i][1]*2 + 1)
        x2, y2 = (soltn_nodes[i+1][0]*2 + 1, soltn_nodes[i+1][1]*2 + 1)
        #Calculate the position of the edge between both nodes
        xO, yO = (x1 + (x2 - x1)//2, y1 + (y2 - y1)//2)
        #Output the first node and edges
        ms_b[x1][y1], ms_g[x1][y1], ms_r[x1][y1] = (0, 255, 0)
        ms_b[xO][yO], ms_g[xO][yO], ms_r[xO][yO] = (0, 255, 0)
    #Output the final node
    ms_b[x2][y2], ms_g[x2][y2], ms_r[x2][y2] = (0, 255, 0)

    #Display starting, ending points
    print("Writing Images...")
    for m_b, m_g, m_r in [(mm_b, mm_g, mm_r),(ms_b, ms_g, ms_r)]:
        for x_offset in range(-1, 2):
            for y_offset in range(-1, 2):
                pos_s = [2*start[0] + 1 + x_offset, 2*start[1] + 1 + y_offset]
                pos_e = [2*end[0]   + 1 + x_offset, 2*end[1]   + 1 + y_offset]
                m_b[pos_s[0]][pos_s[1]], m_g[pos_s[0]][pos_s[1]], m_r[pos_s[0]][pos_s[1]] = (255, 0, 0)
                m_b[pos_e[0]][pos_e[1]], m_g[pos_e[0]][pos_e[1]], m_r[pos_e[0]][pos_e[1]] = (0, 0, 255)
    maze_plain = cv2.merge([mm_b, mm_g, mm_r])
    maze_soltn = cv2.merge([ms_b, ms_g, ms_r])
    
    #Generate a color image overlay
    si_b, si_g, si_r = cv2.split(scaled_img)
    for x in range(len(si_b)):
        for y in range(len(si_b[0])):
            si_b[x][y] = min(si_b[x][y], maze_mask[x][y])
            si_g[x][y] = min(si_g[x][y], maze_mask[x][y])
            si_r[x][y] = min(si_r[x][y], maze_mask[x][y])
    maze_color = cv2.merge([si_b, si_g, si_r])
    
    #Return the maze, the starting point, and the ending point
    print("Done!")
    return maze_plain, maze_soltn, maze_color

In [6]:
def adventure_mode(nodes, edges, soltn_nodes):
    #Get starting and ending position, set player position as start position
    start, player_pos, end = [soltn_nodes[0], soltn_nodes[0], soltn_nodes[-1]]
    #User selection
    sel = ""
    print("Thank you for choosing adventure mode! Good luck if this is a large maze...\n")
    #While the player hasn't beaten the maze or decided to exit
    while player_pos != end and sel not in ['e', 'E']:
        print("You are at location", player_pos, "\nYour destination is at", end)
        
        #Find user selection
        poss = []
        for edge in edges:
            if player_pos in edge:
                #Get the next option
                poss.append(edge[1] if player_pos == edge[0] else edge[0])
                
                #Output option (including direction) to user
                d = (('West'  if poss[-1][0] < player_pos[0] else 'East')   if poss[-1][0] != player_pos[0]
                else ('North' if poss[-1][1] < player_pos[1] else 'South'))
                print("Type", str(len(poss) - 1), "to travel", d, "to", poss[-1])
                
        #Get user response and handle errors
        try:
            sel = input("Choose your move: ")
            #If the user isn't trying to quit
            if sel not in ['e','E']:
                #Try to assign new position
                player_pos = poss[int(sel)]
                if player_pos == end:
                    print("You did it! You beat the videogame!")
        except:
            print("Not a valid location. Type e to exit.")
        
        #Output newline
        print("")

In [7]:
def maze_from_input_filename(filename, info="", size=100, thres=60, noise=0, kernel_size=(3, 3),
                             stack_size_thres=-1, selections=100,
                             equalize_hist=False, print_path_prog=True, adventure=False, seed="Maze Generator"):
    print("Generating images for " + filename.split('.')[0] + info + "...")
    #Scale the image
    scaled_img, scaled_gray_img = scale_for_maze(filename, size)
    #Perform Histogram Equalization if requested
    scaled_gray_img = cv2.equalizeHist(scaled_gray_img) if equalize_hist else scaled_gray_img
    
    start_time = t.time()
    nodes, edges, soltn_nodes = generate_path(scaled_gray_img, thres, noise=noise, kernel_size=kernel_size,
                                              stack_size_thres=stack_size_thres, selections=selections,
                                              print_path_prog=print_path_prog,seed=seed)
    
    total_pth_gen_time = round(t.time() - start_time, 2)
    
    start_time = t.time()
    maze_plain, maze_soltn, maze_color = get_maze_images(scaled_img, nodes, edges, soltn_nodes)
    total_img_gen_time = round(t.time() - start_time, 2)
    
    cv2.imwrite("output/" + filename.split('.')[0] + info + "_maze_plain.jpg", maze_plain)
    cv2.imwrite("output/" + filename.split('.')[0] + info + "_maze_soltn.jpg", maze_soltn)
    cv2.imwrite("output/" + filename.split('.')[0] + info + "_maze_color.jpg", maze_color)
    
    print("The algorithm took " + str(total_pth_gen_time)                              + "sec to generate the path and")
    print("                   " + str(total_img_gen_time)                              + "sec to generate the images.")
    print("   Took a total of " + str(round(total_pth_gen_time+total_img_gen_time, 2)) + "sec to generate the maze.")
    print("\n\n\n")
    
    #Enter adventure mode
    if adventure:
        adventure_mode(nodes, edges, soltn_nodes)
        
    return nodes, edges, soltn_nodes

In [11]:
def main():
    maze_from_input_filename("pointbuilding.jpg",thres=90,size=200,print_path_prog=False)
    maze_from_input_filename("treefrog.jpg", info="Large", size=500, noise=0.05, equalize_hist=True, thres=50)
    maze_from_input_filename("treefrog.jpg", info="Tiny", size=30, noise=0.02,equalize_hist=True, thres=50)
    return maze_from_input_filename("treefrog.jpg", info="Small", size=75, equalize_hist=True, thres=50)

global_nes = main()

Generating images for pointbuilding...
5372 of 5558 nodes were reachable by the algorithm.


Writing edges...: 100%|████████████████████████████████████████████████████████| 5370/5370 [00:00<00:00, 657349.19it/s]


Finding Best Solution...
Writing Images...
Done!
The algorithm took 0.15sec to generate the path and
                   0.1sec to generate the images.
   Took a total of 0.25sec to generate the maze.




Generating images for treefrogLarge...
Visited 1000 nodes of  120451 possibly valid nodes.
Visited 2000 nodes of  120451 possibly valid nodes.
Visited 3000 nodes of  120451 possibly valid nodes.
Visited 4000 nodes of  120451 possibly valid nodes.
Visited 5000 nodes of  120451 possibly valid nodes.
Visited 6000 nodes of  120451 possibly valid nodes.
Visited 7000 nodes of  120451 possibly valid nodes.
Visited 8000 nodes of  120451 possibly valid nodes.
Visited 9000 nodes of  120451 possibly valid nodes.
Visited 10000 nodes of  120451 possibly valid nodes.
Visited 11000 nodes of  120451 possibly valid nodes.
Visited 12000 nodes of  120451 possibly valid nodes.
Visited 13000 nodes of  120451 possibly valid nodes.
Visited 14000 nodes of  120451 possibly valid nodes.
Visited 15000 nodes of  

Writing edges...: 100%|████████████████████████████████████████████████████| 126111/126111 [00:00<00:00, 464247.14it/s]


Finding Best Solution...
Writing Images...
Done!
The algorithm took 8.91sec to generate the path and
                   1.66sec to generate the images.
   Took a total of 10.57sec to generate the maze.




Generating images for treefrogTiny...
180 of 180 nodes were reachable by the algorithm.


Writing edges...: 100%|██████████████████████████████████████████████████████████| 183/183 [00:00<00:00, 184731.08it/s]


Finding Best Solution...
Writing Images...
Done!
The algorithm took 0.01sec to generate the path and
                   0.01sec to generate the images.
   Took a total of 0.02sec to generate the maze.




Generating images for treefrogSmall...
Visited 1000 nodes of  1191 possibly valid nodes.
1191 of 1191 nodes were reachable by the algorithm.


Writing edges...: 100%|████████████████████████████████████████████████████████| 1189/1189 [00:00<00:00, 594472.22it/s]

Finding Best Solution...
Writing Images...
Done!
The algorithm took 0.03sec to generate the path and
                   0.03sec to generate the images.
   Took a total of 0.06sec to generate the maze.






In [9]:
nodes, edges, soltn_nodes = global_nes
adventure_mode(nodes, edges, soltn_nodes)

Thank you for choosing adventure mode! Good luck if this is a large maze...

You are at location (0, 20) 
Your destination is at (36, 0)
Type 0 to travel North to (0, 19)
Type 1 to travel East to (1, 20)
Type 2 to travel South to (0, 21)
Choose your move: e

